In [ ]:
import os
import numpy
numpy.random.seed(33)
numpy.set_printoptions(threshold=numpy.inf,linewidth=300,precision=2)
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.float_format',lambda x:'%.4f'%x)
from PIL import Image
import matplotlib.image
import tensorflow
import keras
from keras.datasets import mnist
from keras.utils import np_utils
from keras.optimizers import SGD,Adagrad,RMSprop,Adam
from keras.models import Sequential,load_model
from keras.callbacks import ModelCheckpoint,TensorBoard,LearningRateScheduler
from keras.layers import InputLayer,Dense,Convolution3D,MaxPooling3D,AveragePooling3D,Flatten,Dropout

In [ ]:
# path
trainPath="../../trainQaq/trainSource"
testPath="../../trainQaq/testSource"

In [ ]:
# assign labels to each name
temp=os.listdir(trainPath)
nameList=[]
for each in temp:
    index=each.find('_')
    nameList.append(each[:index])
nameList=numpy.unique(nameList)
numberList=list(range(0,len(nameList)))
lableDict={}
for i in range(0,len(nameList)):
    lableDict[nameList[i]]=numberList[i]

In [ ]:
# load data
def loadData(path, lableDict):
    imageList=os.listdir(path)
    dataList=[]
    lableList=[]
    num=len(imageList)
    count=0
    i=1
    for each in imageList:
        count+=1
        im=matplotlib.image.imread(path+"\\"+each) # read as array
        im=Image.fromarray(numpy.uint8(im)) # convert to image
        im_resize=im.resize((256, 256)) # compress image to 256*256 (width*height)
        im_resize=numpy.array(im_resize) # convert image to array
        im_resize=im_resize.reshape(-1, 1) # "stretch" the array into one row 
        im_resize=im_resize[:256*256*3]
        im_resize=im_resize.reshape(256,256,3)
        dataList.append(im_resize) # append to data list
        nameLength=each.find("_") # find the length of the name
        lableList.append(lableDict[each[:nameLength]]) # number each labal
        # progress of loading
        if count%50==0 or count==0:
            i=1
        else:
            i+=1
        finished="="*i
        dots="."*(51-i)
        complement="="*(50-num%50)
        if count==num:
            if num%50==0:
                print("\rreading {}[{}=]{}/{} | finished\n".format(path,finished,count,num),end="")
            else:
                print("\rreading {}[{}={}]{}/{} | finished\n".format(path,finished,complement,count,num),end="")
        else:
            print("\rreading {}[{}>{}]{}/{}".format(path,finished,dots,count,num),end="")
    return numpy.array(dataList), numpy.array(lableList)        

In [ ]:
# load data
testIm,testLable=loadData(testPath,lableDict)
trainIm,trainLable=loadData(trainPath,lableDict)

In [ ]:
# one-hot coding
trainLableCoded=np_utils.to_categorical(trainLable,len(numberList))
testLableCoded=np_utils.to_categorical(testLable,len(numberList))
trainImFloat=trainIm.astype("float32")
testImFloat=testIm.astype("float32")
# maximum bightness is 255，after to division, its correct to [0,1]
trainImFloat/=255
testImFloat/=255
# change the shape
trainImFloat=trainImFloat.reshape(-1,1,256,256,3)
testImFloat=testImFloat.reshape(-1,1,256,256,3)



# shuffle
index=list(range(len(trainLableCoded)))
numpy.random.shuffle(index)
trainImFloat=trainImFloat[index]
trainLableCoded=trainLableCoded[index]

In [ ]:
model_CNN = Sequential()
model_CNN.add(InputLayer(input_shape=(1,256,256,3)))
model_CNN.add(MaxPooling3D(pool_size=(3,3,3), padding='same'))
model_CNN.add(Convolution3D(filters=16, kernel_size=(1,3,5), strides=(2,2,2), padding='same',activation='relu'))
model_CNN.add(Convolution3D(filters=16, kernel_size=(3,1,2), strides=(2,2,2), padding='same',activation='relu'))
model_CNN.add(Convolution3D(filters=10, kernel_size=(5,3,2), strides=(1,1,1), padding='same',activation='relu'))
model_CNN.add(MaxPooling3D(pool_size=(3,3,3), padding='same'))
model_CNN.add(Dropout(0.05))
model_CNN.add(Convolution3D(filters=10, kernel_size=(3,5,3), strides=(1,1,1), padding='same',activation='relu'))
model_CNN.add(MaxPooling3D(pool_size=(2,2,2), padding='same'))
model_CNN.add(Dropout(0.08))
model_CNN.add(Convolution3D(filters=8, kernel_size=(3,3,3), strides=(1,1,1), padding='same',activation='relu'))
model_CNN.add(Dropout(0.08))
model_CNN.add(MaxPooling3D(pool_size=(3,1,3), padding='same'))
model_CNN.add(Dropout(0.05))
model_CNN.add(Flatten())
model_CNN.add(Dense(64,activation='relu'))
model_CNN.add(Dense(17,activation='softmax'))
opt = Adam(learning_rate=0.0012)
model_CNN.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
tensorboard = TensorBoard(log_dir="../../bestModel/log")
bestModel="../../bestModel/model_CNN.best.h5"
checkpoint=ModelCheckpoint(bestModel,monitor="val_loss",verbose=0,mode='min')
keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=6, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
callbacks_list=[tensorboard,checkpoint]
batch = [8]*10
count = 0
for batch_size in batch:
    count+=1
    print('Training......',count)
    model_CNN.fit(trainImFloat,trainLableCoded,batch_size=batch_size,epochs=5,callbacks=callbacks_list)
    print('Testing......')
    loss,accuracy = model_CNN.evaluate(testImFloat, testLableCoded)
    print('test loss = ',loss)
    print('test accuracy = ',accuracy)

In [ ]:
model_CNN.save('../../bestModel/model_CNN.h5')

In [ ]:
model_CNN=load_model('../../bestModel/model_CNN.h5')

In [ ]:
model_CNN=load_model('../../bestModel/model_CNN.best.h5')

In [ ]:
testNum=numpy.random.randint(1,testIm.shape[0])
im=Image.fromarray(numpy.uint8(testIm[testNum]))
im.show()

In [ ]:
prediction=list(model_CNN.predict(testImFloat[[testNum],])[0])
print('result: ',nameList[prediction.index(max(prediction))])
prediction_dataframe = pd.DataFrame({'name':nameList,'posibility':prediction})
prediction_dataframe